# Planet Image Search and MODIS Cloud Cover Filter
### This script searches Planet PSScene images and filters the search results by cloud cover using MODIS and Planet derived cloud cover metrics.
#### TODO:
- figure out how to ensure coverage in all cells

## API and Package Set-Up

In [ ]:
# Start Earth Engine API
import ee
ee.Initialize()

In [ ]:
# Import Libraries
import geemap
import os
import json
import requests
from requests.auth import HTTPBasicAuth
import urllib.request
from PIL import Image
from io import BytesIO
import numpy as np
import pandas as pd
import geopandas as gpd
import shapely as shp
from pprint import pprint
import ast
import matplotlib.pyplot as plt
import seaborn as sns
import re
import warnings

In [ ]:
# Get Planet API Key
%load_ext dotenv
%dotenv

api_key = os.getenv('PL_API_KEY')

## Data Import

In [ ]:
# Import MODIS Data
modis = ee.ImageCollection('MODIS/061/MOD09GA');

# MODIS snow
modis_snow = ee.ImageCollection('MODIS/006/MOD10A1')

In [ ]:
# Import shapefile with AOI (multipolygon)
aoi = gpd.read_file("/home/hrodenhizer/Documents/permafrost_pathways/rts_mapping/planet_processing_test/data/yg_val_regions/bboxes/yg_validation_bboxes.shp")
# convert from multipolygon to multiple polygons
aoi = aoi.explode(column = 'geometry', ignore_index = True)
# remove inner holes
aoi.geometry = aoi.geometry.exterior
# convert back to polygon
aoi.geometry = [shp.geometry.Polygon([shp.geometry.Point(x, y) for x, y in list(feature.coords)]) for feature in aoi.geometry]
aoi['region'] = aoi.index
# convert to json for planet data search
sites = json.loads(aoi.to_json()) # if multiple sites

In [ ]:
aoi

In [ ]:
# Years to test
years = [2017, 2018, 2019, 2020, 2021]

## Define Functions

In [ ]:
# define function to extract specific bits from bitmask
def bitwiseExtract(input, fromBit, toBit):
    maskSize = ee.Number(1).add(toBit).subtract(fromBit)
    mask = ee.Number(1).leftShift(maskSize).subtract(1)
    return input.rightShift(fromBit).bitwiseAnd(mask)

In [ ]:
# define function to get snow data from MODIS
def modisSnow(modis_snow_imagery, date, aoi):
    # extract NDSI
    snow_cover = (ee.Image(modis_snow_imagery
                           .filter(ee.Filter.date(date))
                           .select(['NDSI_Snow_Cover'])
                           .first())
                  .clip(aoi));
    
    # get average snow cover
    snow_cover = snow_cover.reduceRegion(ee.Reducer.max(), aoi);
    
    return snow_cover.getInfo()

In [ ]:
# function to figure out UTM zone from lon
def long2UTM(long):
    zone = int(np.floor((long + 180)/6) % 60) + 1
    return 'EPSG:'+ str(32600 + zone) # would be 32700 for Southern Hemisphere

In [ ]:
# define function to extract metadata needed for cloud calculation
def getMetadata(feature, aoi, region, crop_aoi = None):
    
    # crop_aoi = aoi, if no crop_aoi provided
    if crop_aoi is None:
        crop_aoi = aoi
    
    # get image id
    img_id = feature['id']
    
    # get image date
    img_date = feature['properties']['acquired'].split('T')[0]
    
    # get instrument type
    instrument_type = feature['properties']['instrument']
    
    # get planet cloud/cloud shadow cover
    
    try:
        img_cloud_cover = float(100 - feature['properties']['clear_percent'])
    except KeyError:
        img_cloud_cover = float(feature['properties']['cloud_cover']*100)
    
    # use intersection of aoi and search result geometry to get actual geometry of cells with data
    img_geometry = (
        aoi[aoi.region == region].geometry
        .intersection(
            shp.geometry.Polygon(
                tuple([(feature[0], feature[1]) for feature in feature['geometry']['coordinates'][0]])
            )
        )
        .reset_index(drop = True)
    )
    img_geometry_crop = (
        crop_aoi[crop_aoi.region == region].geometry
        .intersection(
            shp.geometry.Polygon(
                tuple([(feature[0], feature[1]) for feature in feature['geometry']['coordinates'][0]])
            )
        )
        .reset_index(drop = True)
    )
    
    # get image area and coverage
    if not img_geometry[0].is_empty:
        # Determine crs
        if str(type(img_geometry[0])) == "<class 'shapely.geometry.polygon.Polygon'>":
            crs = long2UTM(list(img_geometry.exterior[0].coords)[0][0])
        elif str(type(img_geometry[0])) == "<class 'shapely.geometry.multipolygon.MultiPolygon'>":
            crs = long2UTM(list(img_geometry[0].geoms)[0].exterior.coords[0][0])
        elif str(type(img_geometry[0])) == "<class 'shapely.geometry.collection.GeometryCollection'>":
            polygon_idx = [item for item in list(img_geometry[0].geoms) if str(type(item)) == "<class 'shapely.geometry.polygon.Polygon'>"]
            crs = long2UTM(list(img_geometry[0].geoms)[0].exterior.coords[0][0])
    # calculate area and % coverage in correct crs
    if img_geometry[0].is_empty or str(type(img_geometry[0])) == "<class 'shapely.geometry.linestring.LineString'>":
        crs = None
        img_area = 0.0
        img_coverage = 0
    else:
        img_area = float(img_geometry_crop.to_crs(crs = crs).area/1e6)
        img_coverage = round(float(img_geometry.to_crs(crs = crs).area)/
                             float(aoi[aoi.region == region].geometry.to_crs(crs = crs).area)*100)
    
    return [img_id, img_date, instrument_type, img_coverage, img_area, img_cloud_cover, img_geometry, crs]

In [ ]:
def geometryToEE(img_geometry, region):
    
    # format geometry
    img_geometry = gpd.GeoDataFrame(geometry = img_geometry)
    img_geometry = [[[x, y] for x, y in list(img_geometry.geometry.exterior[0].coords)]]
    
    # convert geometry to ee.Geometry
    img_geometry_ee = ee.Geometry({
        'type': 'Polygon',
        'coordinates': img_geometry
    })
    
    return img_geometry_ee

In [ ]:
# define function to calculate MODIS cloud cover
def modisCloudCover(modis_imagery, date, aoi):
    # extract QC bitmask band from MODIS
    qc = (ee.Image(modis_imagery
              .filter(ee.Filter.date(date))
              .first())
      .select(['state_1km'])
      .clip(aoi));
    
    # extract cloud information from MODIS QC bitmask
    cloud_mask = bitwiseExtract(qc, 0, 1).remap([0, 1, 2, 3], [0, 1, 1, 1])
    area_mask = cloud_mask.remap([0, 1], [1, 1])
    
    # calculate area of cells with clouds
    cloud_area_img = cloud_mask.multiply(ee.Image.pixelArea())
    area_img = area_mask.multiply(ee.Image.pixelArea())
    
    # calculate cloud cover percent
    cloud_area = (
        cloud_area_img
        .reduceRegion(
            reducer = ee.Reducer.sum(), # calculate total cloud area
            geometry = aoi,
            scale = 1000,
            maxPixels = 1e10
        )
        .getNumber('remapped')
        .divide(
            area_img
            .reduceRegion(
                reducer = ee.Reducer.sum(), # calculate total area
                geometry = aoi,
                scale = 1000,
                maxPixels = 1e10
            )
            .getNumber('remapped')
        ) # divide cloud area by total area
        .multiply(ee.Number(100)) # convert to %
        .round() # remove decimal precision
    );
    
    return cloud_area.getInfo()

In [ ]:
# download with retry
def recu_down(url, filename): # recurrent download with ContentTooShortError
    try:
        urllib.request.urlretrieve(url,filename)
    except urllib.error.ContentTooShortError:
        print('Download failed. Trying again...')
        recu_down(url, filename)
        


## Determine Snow Free Dates

In [ ]:
# modis_snow_df = pd.DataFrame(columns = ['region', 'year', 'date', 'snow_cover'])
# for index, row in aoi.iterrows():
#     print(index)
#     geometry = [[[x, y] for x, y in list(row.geometry.exterior.coords)]]
    
#     # convert geometry to ee.Geometry
#     geometry_ee = ee.Geometry({
#         'type': 'Polygon',
#         'coordinates': geometry
#     })
#     for year in years:
#         print(year)
#         dates = pd.date_range(str(year) + '-06-01', str(year) + '-08-31', freq = 'D')
        
#         for date in dates:
            
#             snow = modisSnow(modis_snow, date, geometry_ee)
            
#             # add to output
#             modis_snow_df = pd.concat([modis_snow_df,
#                                        pd.DataFrame({'region': index,
#                                                      'year': year,
#                                                      'date': date,
#                                                      'snow_cover': snow})])

# modis_snow_df = modis_snow_df.fillna(value = np.NAN)
# modis_snow_df = modis_snow_df.reset_index(drop = True)
# modis_snow_df

In [ ]:
# modis_snow_df.to_csv('/home/hrodenhizer/Documents/permafrost_pathways/rts_mapping/planet_processing_test/data/yg_val_regions/modis_snow_data.csv',
#                     index = False)
modis_snow_df = pd.read_csv('/home/hrodenhizer/Documents/permafrost_pathways/rts_mapping/planet_processing_test/data/yg_val_regions/modis_snow_data.csv')
modis_snow_df['date'] = pd.to_datetime(modis_snow_df['date'])
modis_snow_df

In [ ]:
# Define first snow-free date as first of three consecutive NDSI == 0 (removing NaN values)
modis_snow_df['snow_free'] = np.where(modis_snow_df.snow_cover == 0, 1, 0)
modis_snow_df['rolling_snow_free'] = (
    modis_snow_df
    .groupby(['region', 'year'])
    .snow_free
    .rolling(2).sum().shift(-1)
    .reset_index(drop = True)
)
snow_free_date = (
    modis_snow_df[modis_snow_df.rolling_snow_free == 2]
    .groupby(['region', 'year'])
    .first()
    .rename(columns = {'date': 'snow_free_date'})
    .snow_free_date
)

snow_free_date

In [ ]:
# import warnings
with warnings.catch_warnings(): # there is a warning getting triggered inside of sns, I think
    warnings.simplefilter("ignore")
    g = sns.FacetGrid(data = modis_snow_df,
                          col = 'year',
                          row = 'region',
                          sharex = False)
    g.map(sns.lineplot, 'date', 'snow_cover')
    
    for ax, pos in zip(g.axes.flat, snow_free_date):
        ax.axvline(x=pos, color='black', linestyle=':')


## Search Planet Imagery

In [ ]:
# all_metadata = []
# metadata_df = pd.DataFrame(columns = ['region', 'year', 'page', 'metadata'])

# # Data type
# item_type = "PSScene"

# # asset filter
# asset_filter = {
#     'type': 'OrFilter',
#     'config': [
#        {
#            "type": "AndFilter",
#             "config": [
#                 {
#                     "type": "AssetFilter",
#                     "config": [
#                         "ortho_analytic_4b_sr"
#                     ]
#                 },
#                 {
#                     "type": "AssetFilter",
#                     "config": [
#                         "ortho_udm2"
#                     ]
#                 }
#             ]
#         },
#         {
#             "type": "AndFilter",
#             "config": [
#                 {
#                     "type": "AssetFilter",
#                     "config": [
#                         "ortho_analytic_8b_sr"
#                     ]
#                 },
#                 {
#                     "type": "AssetFilter",
#                     "config": [
#                         "ortho_udm2"
#                     ]
#                 }
#             ]
#         } 
#     ]
    
# }

# for region, site in enumerate(sites['features']):
    
#     print('Region: ' + site['id'])
#     site_name = site['id']

#     session = requests.Session()
#     session.auth = (api_key, '')
#     site_coords = site['geometry']['coordinates']

#     site_dict = {
#         "type": "Polygon",
#         "coordinates": site_coords}

#     # get images that overlap with our aoi
#     geometry_filter = {
#         "type": "GeometryFilter",
#         "field_name": "geometry",
#         "config": site_dict
#     }

#     for year in years:
        
#         if snow_free_date.loc[(region, year)] < pd.to_datetime('{}-07-01'.format(year)):
#             start_date = str(snow_free_date.loc[(region, year)])[0:10]
#         else:
#             start_date = '{}-07-01'.format(year)

#         # i only want images between these two dates of each year...easier to search within a year to avoid massive search queries
#         start_date = "{}T00:00:00.000Z".format(start_date)
#         end_date = "{}-09-30T00:00:00.000Z".format(year)

#         # get images acquired within a date range
#         date_range_filter = {
#             "type": "DateRangeFilter",
#             "field_name": "acquired",
#             "config": {
#                 "gte": start_date,
#                 "lte": end_date
#             }
#         }

#         cloud_cover_filter = {
#             "type": "RangeFilter",
#             "field_name": "cloud_cover",
#             "config": {
#                 "lte": 1 # cloud cover threshold - none currently
#             }
#         }

#         combined_filter = {
#             "type": "AndFilter",
#             "config": [
#                 asset_filter,
# #                 instrument_filter,
#                 geometry_filter,
#                 date_range_filter,
#                 cloud_cover_filter
#             ]
#         }

#         # API request object
#         search_request = {
#             "item_types": [item_type],
#             "filter": combined_filter
#         }

#         # fire off the POST request
#         search_result = \
#           requests.post(
#             'https://api.planet.com/data/v1/quick-search',
#             auth=HTTPBasicAuth(api_key, ''),
#             json=search_request)

#         all_metadata.append(search_result.json())
        
#         page = 1
#         print('Page: ' + str(page))
        
#         # format metadata for dataframe
#         temp_df = pd.DataFrame({'region': site_name,
#                                 'year': year,
#                                 'page': page,
#                                 'metadata': [search_result.json()]})
        
#         metadata_df = pd.concat([metadata_df, temp_df], axis = 0)
#         print('Results from page ' + str(page) + ' added.')

#         try:
#             next_link = search_result.json()['_links']['_next']
            
#             while next_link:
#                 page = page + 1
#                 print('Page: ' + str(page))
#                 search_result = \
#                   requests.get(
#                     next_link,
#                     auth=HTTPBasicAuth(api_key, ''))
                
#                 all_metadata.append(search_result.json())

#                 # format metadata for dataframe
#                 temp_df = pd.DataFrame({'region': site_name,
#                                         'year': year,
#                                         'page': page,
#                                         'metadata': [search_result.json()]})
#                 metadata_df = pd.concat([metadata_df, temp_df], axis = 0)
#                 print('Results from page ' + str(page) + ' added.')

#                 try:
#                     next_link = search_result.json()['_links']['_next']
#                 except:
#                     next_link = None
#         except:
#             next_link = None
            
#     print('\n')
            
# metadata_df = metadata_df.reset_index(drop = True)
# metadata_df

In [ ]:
# # save output
# metadata_df.to_csv('/home/hrodenhizer/Documents/permafrost_pathways/rts_mapping/planet_processing_test/data/yg_val_regions/planet_image_search.csv')
metadata_df = pd.read_csv('/home/hrodenhizer/Documents/permafrost_pathways/rts_mapping/planet_processing_test/data/yg_val_regions/planet_image_search.csv',
                          index_col=0,
                          converters = {'metadata': ast.literal_eval})
metadata_df

## Calculate Cloud Cover

In [ ]:
# cloud_data = pd.DataFrame(
#     columns = [
#         'region', 
#         'year', 
#         'date', 
#         'id',
#         'instrument',
#         'coverage', 
#         'area', 
#         'modis_cloud_cover', 
#         'planet_cloud_cover', 
#         'cloud_cover', 
#         'modis_planet_diff',
#         'geometry']
# )
# for index,row in metadata_df.iterrows():
    
#     print(index);
    
#     # get metadata
#     region = int(row['region']);
#     img_year = int(row['year']);
#     metadata = row['metadata']['features'];
    
#     if len(metadata) > 0: 
#         for feature in metadata:
            
#             # extract metadata needed for cloud cover calculations
#             img_id, img_date, instrument_type, img_coverage, img_area, img_cloud_cover, img_geometry, crs = getMetadata(feature, aoi, region);
# #             print(img_id)
            
#             # calculate modis cloud cover if the geometry is a polygon
#             if str(type(img_geometry[0])) == "<class 'shapely.geometry.polygon.Polygon'>":
#                 # convert geometry to ee.Geometry
#                 img_geometry_ee = geometryToEE(img_geometry, region);

#                 # calc modis cloud cover
#                 modis_cloud_cover = modisCloudCover(modis, img_date, img_geometry_ee);
#                 if modis_cloud_cover > 100:
#                     modis_cloud_cover = 100;
                
#             # calculate modis cloud cover if the geometry is a multipolygon
#             elif str(type(img_geometry[0])) == "<class 'shapely.geometry.multipolygon.MultiPolygon'>":
                
#                 polygon_geometries = gpd.GeoDataFrame(
#                     geometry = img_geometry
#                 ).explode(column = 'geometry', ignore_index = True)
#                 modis_cloud_cover = []
#                 polygon_area = []
#                 for index, row in polygon_geometries.iterrows():
                    
#                     # calculate area of sub polygon
#                     temp_area = gpd.GeoDataFrame(
#                         geometry = row, 
#                         crs = polygon_geometries.crs
#                     ).reset_index().to_crs(crs = crs).geometry.area/1e6
#                     polygon_area.append(temp_area)
                    
#                     # convert geometry to ee.Geometry
#                     polygon_geometry = [[[x, y] for x, y in list(row.geometry.exterior.coords)]]
    
#                     # convert geometry to ee.Geometry
#                     polygon_geometry_ee = ee.Geometry({
#                         'type': 'Polygon',
#                         'coordinates': polygon_geometry
#                     })

#                     # calc modis cloud cover of sub polygon
#                     temp_cloud_cover = modisCloudCover(modis, img_date, polygon_geometry_ee);
#                     modis_cloud_cover.append(temp_cloud_cover)
                    
#                 # calculate average cloud cover of multipolygon (img_geometry)
#                 modis_cloud_cover = int(round(sum(
#                     [cloud * area for cloud, area in zip(modis_cloud_cover, polygon_area)]
#                 )/img_area))
#                 if modis_cloud_cover > 100:
#                     modis_cloud_cover = 100;
                    
#             # get higher cloud cover estimate
#             cloud_cover = np.maximum(modis_cloud_cover, img_cloud_cover)
            
#             # calculate difference between cloud cover estimates
#             modis_planet_diff = abs(
#                 modis_cloud_cover - img_cloud_cover
#             )
            
#             # organize data into a dataframe
#             temp_df = pd.DataFrame({
#                 'region': region,
#                 'year': img_year,
#                 'date': img_date,
#                 'id': img_id,
#                 'instrument': instrument_type,
#                 'coverage': img_coverage,
#                 'area': img_area,
#                 'modis_cloud_cover': modis_cloud_cover,
#                 'planet_cloud_cover': img_cloud_cover,
#                 'cloud_cover': cloud_cover,
#                 'modis_planet_diff': modis_planet_diff,
#                 'geometry': img_geometry
#             });

#             # append new data to old
#             cloud_data = pd.concat([cloud_data, temp_df], axis = 0);

# cloud_data = gpd.GeoDataFrame(cloud_data);
# cloud_data


In [ ]:
# # save output
# cloud_data.to_csv('/home/hrodenhizer/Documents/permafrost_pathways/rts_mapping/planet_processing_test/data/yg_val_regions/planet_images_modis_cloud.csv')
cloud_data = pd.read_csv('/home/hrodenhizer/Documents/permafrost_pathways/rts_mapping/planet_processing_test/data/yg_val_regions/planet_images_modis_cloud.csv',
                  index_col = 0)
cloud_data = gpd.GeoDataFrame(cloud_data,
                              geometry = gpd.GeoSeries.from_wkt(cloud_data['geometry']),
                              crs = 'EPSG:4326')
cloud_data['date'] = pd.to_datetime(cloud_data['date'])
cloud_data

In [ ]:
sns.displot(data = cloud_data, 
            x = 'area',
#             hue = 'year',
#             multiple = 'stack',
#             alpha = 0.5,
           )

## Filter Images on Cloud Cover and Date

In [ ]:
# get count of images by polygon and year
image_counts = cloud_data[['region', 'year']].value_counts()
image_counts = pd.DataFrame(image_counts, columns = ['count']).sort_index()
# image_counts.to_csv('/home/hrodenhizer/Documents/permafrost_pathways/rts_mapping/planet_processing_test/data/yg_val_regions/image_counts_pre_filter.csv')
image_counts

In [ ]:
# filter by area >= 10 km^2 to ensure at least a few tie points for AROSICS
# Don't filter on cloud cover yet?
potential_images = cloud_data[cloud_data['area'] >= 10]
potential_images

In [ ]:
uncertain_imgs = pd.DataFrame(potential_images[(potential_images['modis_planet_diff'] > 80) 
                                               & (potential_images['planet_cloud_cover'] < 20)].id)
uncertain_imgs['keep'] = None
uncertain_imgs

In [ ]:
# get a list of thumbnail urls
thumbnails = [
    feature['_links']['thumbnail'] 
    for search in metadata_df.metadata 
    for feature in search['features'] 
    if feature['id'] in [
        str(image) 
        for image in potential_images[(potential_images['modis_planet_diff'] > 80) 
                                      & (potential_images['planet_cloud_cover'] < 20)].id
 ]
]
print(len(thumbnails))
def remove_duplicates(sequence):
    lst = []
    for i in sequence:
        if i not in lst:
            lst.append(i)
    # returns unsorted unique list
    return lst

thumbnails = remove_duplicates(thumbnails)
print(len(thumbnails))
thumbnails

In [ ]:
# # preview thumbnails to inspect
# for link in thumbnails:
#     img_id = link.split('/')[8]
#     print('Viewing thumbnail for ' + str(img_id))
#     response = requests.get(link,
#                     auth=HTTPBasicAuth(api_key, ''))
#     thumbnail = Image.open(BytesIO(response.content))
#     plt.imshow(thumbnail)
#     plt.show()
    
#     # provide input about whether to download the image
#     keep = input('Do you want to download this image?(y/n)\n')
    
#     # save user input with img_id to use as a filter later
#     uncertain_imgs.loc[uncertain_imgs.id == str(img_id), 'keep'] = keep
#     print(uncertain_imgs.loc[uncertain_imgs.id == str(img_id)])

In [ ]:
# uncertain_imgs.to_csv('/home/hrodenhizer/Documents/permafrost_pathways/rts_mapping/planet_processing_test/data/yg_val_regions/uncertain_images.csv',
#                       index = False)
uncertain_imgs = pd.read_csv('/home/hrodenhizer/Documents/permafrost_pathways/rts_mapping/planet_processing_test/data/yg_val_regions/uncertain_images.csv')
uncertain_imgs

In [ ]:
potential_images = pd.merge(potential_images,
                            uncertain_imgs,
                            how = "outer",
                            on = 'id')
potential_images

In [ ]:
# use planet_cloud_cover number in cases where high MODIS cloud cover has been determined to be incorrect
potential_images.loc[(potential_images.modis_planet_diff > 80) 
                     & (potential_images.planet_cloud_cover < 20)
                     & (potential_images.keep == 'y'),
    'cloud_cover'
] = potential_images.planet_cloud_cover
potential_images.loc[potential_images.keep == 'y']

In [ ]:
# create list of dates in order of preference which will be used to sort the dataframe so that we can slice it to get image dates
date_order = pd.DataFrame()
for polygon_id in aoi.index:
    for year in potential_images['year'].unique():
        if snow_free_date[polygon_id, year] <= pd.Timestamp(str(year) + '-07-01'):
            start_date = snow_free_date[polygon_id, year]
        else:
            start_date = pd.Timestamp(str(year) + '-07-01')
        dates = pd.DataFrame({'date': pd.date_range(start = start_date,
                                                    end = str(year) + '-08-31',
                                                    freq = 'D').strftime('%Y-%m-%d')})
        split_loc = np.where([bool(re.search('08-01', date)) for date in dates.date])[0][0]
        dates_1 = np.flip(np.arange(0, split_loc))
        dates_2 = np.arange(split_loc, len(dates))
        idx = list(np.insert(dates_1, np.arange(0, len(dates_2)), dates_2))
        dates = dates.iloc[idx].reset_index(drop = True).rename(columns = {0: 'date'})
        dates['polygon_id'] = polygon_id
        dates['year'] = year
        dates['idx'] = dates.index.astype('Int32')
        dates['date'] = pd.to_datetime(dates['date'])
        dates = dates.set_index(['polygon_id', 'year', 'date'])
        date_order = pd.concat([date_order, dates])
date_order

In [ ]:
# filter images based on clouds and dates
images_ordered = pd.DataFrame(columns = potential_images.columns)
for region in aoi.index:
    print('###################################################')
    print(region)
    print('###################################################')
    
    polygon_geometry = (
        aoi[region:region+1].reset_index()
        .rename(columns = {'index': 'region'})
        .loc[:, ['region','geometry']]
    )
    
    for year in potential_images['year'].unique():
        print(year)
        
        # get data
        temp_data = potential_images[(potential_images['region'] == region) &
                                     (potential_images['year'] == year)]
        
        # first get all images with no clouds
        temp_output = (
            temp_data[(temp_data['cloud_cover'] == 0)]
            # arrange in correct date order
            .join(date_order, on = ['region', 'year', 'date'])
            .sort_values(by = ['idx'])
            .reset_index(drop = True)
        )
        
        ### It would be nice to get an image count across the entire image and make sure each location
        ### has at least 10, but I haven't been able to figure out how
#         # check number of images
#         polygon_geometry['n_images'] = 0
#         n_images = temp_output.loc[:, ['region', 'geometry']]
#         n_images['n_images'] = 1
#         n_images = pd.concat([n_images, polygon_geometry])
#         img_union = n_images.overlay(n_images, how = 'union')
        
        if sum(temp_output['coverage']) < 1000:
            cloud_lwr = 0
            cloud_upr = 10
            while sum(temp_output['coverage']) < 1000 and cloud_upr < 50 and len(temp_data) > len(temp_output):
                temp_output = (
                    pd.concat([temp_output,
                               (temp_data[(temp_data['cloud_cover'] > cloud_lwr) &
                                          (temp_data['cloud_cover'] <= cloud_upr)]
                                # arrange in correct date order
                                .join(date_order, on = ['region', 'year', 'date'])
                                .sort_values(by = ['idx'])
                                .reset_index(drop = True))])
                    .reset_index(drop = True)
                )
                
                cloud_lwr = cloud_lwr + 10
                cloud_upr = cloud_upr + 10
                
        print('coverage: ' + str(sum(temp_output['coverage'])))
        print('coverage - 1: ' + str(sum(temp_output.drop([temp_output.tail(1).index[0]])['coverage'])))
        # check coverage - look for 1000% coverage over all images
        while sum(temp_output.drop([temp_output.tail(1).index[0]])['coverage']) > 1000:
            print('Removing index ' + str(temp_output.tail(1).index[0]))
            temp_output = temp_output.drop([temp_output.tail(1).index[0]])
            print('coverage - 1: ' + str(sum(temp_output.drop([temp_output.tail(1).index[0]])['coverage'])))


        # add images to output
        images_ordered = pd.concat([images_ordered, 
                                    temp_output])
        print('\n')

images_ordered.reset_index()
# images_ordered.to_csv('/home/hrodenhizer/Documents/permafrost_pathways/rts_mapping/planet_processing_test/data/yg_val_regions/planet_images_filtered.csv',
#                       index = False)
images_ordered


In [ ]:
# a bit of clean-up
images_ordered['coverage'] = pd.to_numeric(images_ordered['coverage'])
len(images_ordered.id)

In [ ]:
# preview thumbnails to remove remaining cloudy images
thumbnails = [
    feature['_links']['thumbnail'] 
    for search in metadata_df.metadata 
    for feature in search['features'] 
    if feature['id'] in [
        str(image) 
        for image in images_ordered.id
 ]
]
print(len(thumbnails))

thumbnails = remove_duplicates(thumbnails)
print(len(thumbnails))

In [ ]:
# # preview thumbnails to inspect
# cloudy_imgs = []
# for link in thumbnails:
#     img_id = link.split('/')[8]
#     print('Viewing thumbnail for ' + str(img_id))
#     response = requests.get(link,
#                     auth=HTTPBasicAuth(api_key, ''))
#     thumbnail = Image.open(BytesIO(response.content))
#     plt.imshow(thumbnail)
#     plt.show()
    
#     # provide input about whether to download the image
#     keep = input('Should this image be removed?(y/n)\n')
    
#     # save user input with img_id to use as a filter later
#     if keep == 'y':
#         cloudy_imgs.append(img_id)
        
# cloudy_imgs = pd.DataFrame({'cloudy_imgs': cloudy_imgs})
# cloudy_imgs.to_csv('/home/hrodenhizer/Documents/permafrost_pathways/rts_mapping/planet_processing_test/data/yg_val_regions/cloudy_images_to_remove.csv',
#                        index = False)

In [ ]:
# # filter images based on clouds and dates
# images_filtered = pd.DataFrame(columns = potential_images.columns)
# for region in aoi.index:
#     print('###################################################')
#     print(region)
#     print('###################################################')
    
#     polygon_geometry = (
#         aoi[region:region+1].reset_index()
#         .rename(columns = {'index': 'region'})
#         .loc[:, ['region','geometry']]
#     )
    
#     for year in potential_images['year'].unique():
#         print(year)
        
#         # get data
#         temp_data = potential_images[(potential_images['region'] == region) &
#                                      (potential_images['year'] == year) &
#                                      (~potential_images.id.isin(list(cloudy_imgs.cloudy_imgs)))]
        
#         # first get all images with no clouds
#         temp_output = (
#             temp_data[(temp_data['cloud_cover'] == 0)]
#             # arrange in correct date order
#             .join(date_order, on = ['region', 'year', 'date'])
#             .sort_values(by = ['idx'])
#             .reset_index(drop = True)
#         )
        
#         ### It would be nice to get an image count across the entire image and make sure each location
#         ### has at least 10, but I haven't been able to figure out how
# #         # check number of images
# #         polygon_geometry['n_images'] = 0
# #         n_images = temp_output.loc[:, ['region', 'geometry']]
# #         n_images['n_images'] = 1
# #         n_images = pd.concat([n_images, polygon_geometry])
# #         img_union = n_images.overlay(n_images, how = 'union')
        
#         if sum(temp_output['coverage']) < 1000:
#             cloud_lwr = 0
#             cloud_upr = 10
#             while sum(temp_output['coverage']) < 1000 and cloud_upr < 50 and len(temp_data) > len(temp_output):
#                 temp_output = (
#                     pd.concat([temp_output,
#                                (temp_data[(temp_data['cloud_cover'] > cloud_lwr) &
#                                           (temp_data['cloud_cover'] <= cloud_upr)]
#                                 # arrange in correct date order
#                                 .join(date_order, on = ['region', 'year', 'date'])
#                                 .sort_values(by = ['idx'])
#                                 .reset_index(drop = True))])
#                     .reset_index(drop = True)
#                 )
                
#                 cloud_lwr = cloud_lwr + 10
#                 cloud_upr = cloud_upr + 10
                
#         print('coverage: ' + str(sum(temp_output['coverage'])))
#         print('coverage - 1: ' + str(sum(temp_output.drop([temp_output.tail(1).index[0]])['coverage'])))
#         # check coverage - look for 1000% coverage over all images
#         while sum(temp_output.drop([temp_output.tail(1).index[0]])['coverage']) > 1000:
#             print('Removing index ' + str(temp_output.tail(1).index[0]))
#             temp_output = temp_output.drop([temp_output.tail(1).index[0]])
#             print('coverage - 1: ' + str(sum(temp_output.drop([temp_output.tail(1).index[0]])['coverage'])))


#         # add images to output
#         images_filtered = pd.concat([images_filtered, 
#                                     temp_output])
#         print('\n')

# images_filtered.reset_index()
# # images_filtered.to_csv('/home/hrodenhizer/Documents/permafrost_pathways/rts_mapping/planet_processing_test/data/yg_val_regions/planet_images_filtered_manual_cloud_removal.csv',
# #                       index = False)
# # images_filtered


In [ ]:
images_filtered = pd.read_csv('/home/hrodenhizer/Documents/permafrost_pathways/rts_mapping/planet_processing_test/data/yg_val_regions/planet_images_filtered_manual_cloud_removal.csv')

In [ ]:
# a bit of clean-up
images_filtered['coverage'] = pd.to_numeric(images_filtered['coverage'])
len(images_filtered.id)

In [ ]:
# get bboxes for all images to visually inspect coverage
bboxes = [
    feature['geometry']['coordinates'][0]
    for search in metadata_df.metadata 
    for feature in search['features'] 
    if feature['id'] in [
        str(image) 
        for image in images_filtered.id
 ]
]
ids = [
    feature['id']
    for search in metadata_df.metadata 
    for feature in search['features'] 
    if feature['id'] in [
        str(image) 
        for image in images_filtered.id
 ]
]

bboxes = [shp.geometry.Polygon([shp.geometry.Point(x, y) for x, y in feature]) for feature in bboxes]
bboxes = gpd.GeoDataFrame(data = {'img_id': ids},
                          crs = 'EPSG:4326',
                          geometry = bboxes)
# bboxes.to_file('/home/hrodenhizer/Documents/permafrost_pathways/rts_mapping/planet_processing_test/data/yg_val_regions/image_footprints/planet_images_footprints.shp')
bboxes
# len(images_filtered)

In [ ]:
# Number of images that have no cloud cover by polygon and year
image_counts_cloud_free = (
    pd.DataFrame(images_filtered[images_filtered['cloud_cover'] == 0][['region', 'year']]
    .groupby(['region', 'year'])
    .value_counts())
    .rename(columns = {0: 'cloud_free_image_count'})
)
image_counts_cloud_free

In [ ]:
# 1 region has only 9 completely cloud free image in 2018
pprint(image_counts_cloud_free[image_counts_cloud_free['cloud_free_image_count'] == min(image_counts_cloud_free['cloud_free_image_count'])])
pd.DataFrame(image_counts_cloud_free[['cloud_free_image_count']]
             .groupby('year')
             .value_counts()
             .sort_index()).rename(columns = {0: 'region_count'})

In [ ]:
sns.displot(data = image_counts_cloud_free, 
            x = 'cloud_free_image_count',
#             hue = 'year',
#             multiple = 'stack',
#             alpha = 0.5,
            row = 'year'
           )

In [ ]:
# get count of images by region and year
image_counts_f1 = images_filtered[['region', 'year']].value_counts()
image_counts_f1 = pd.DataFrame(image_counts_f1, columns = ['count']).sort_index()
image_counts_f1 = (
    image_counts_f1.join(
        images_filtered[['region', 'year', 'coverage', 'area', 'cloud_cover']]
        .groupby(['region', 'year'])
        .aggregate({'coverage': 'sum',
                    'area': 'sum',
                    'cloud_cover': 'max'})
        .rename(columns = {'coverage': 'cumulative_coverage',
                           'area': 'cumulative_area',
                           'cloud_cover': 'max_cloud_cover'})
    ).rename(columns = {'count': 'img_count'})
)
image_counts_f1

In [ ]:
# 40% is the max cloud cover in an image
image_counts_f1[image_counts_f1['max_cloud_cover'] == max(image_counts_f1['max_cloud_cover'])]

In [ ]:
# Calculate the total area if all of these images are downloaded
images_filtered['planet_quota_usage'] = [area if area >= 100 else 100 for area in images_filtered.area]
total_area = round(sum(images_filtered.area))
print(total_area)
print(round(total_area/5000000*100, ndigits = 2))
quota_usage = round(sum(images_filtered.planet_quota_usage))
print(quota_usage)
print(round(quota_usage/5000000*100, ndigits = 2))

In [ ]:
# determine area that would be used if I didn't clip the images
image_footprints = []
for img_id in images_filtered.id:
    image_footprints.append([shp.geometry.Polygon(feature['geometry']['coordinates'][0]) 
                             for metadata in metadata_df.metadata 
                             for feature in metadata['features']
                             if feature['id'] == img_id][0])
image_footprints = gpd.GeoDataFrame({'geometry': image_footprints},
                                   crs = 'EPSG:4326')
image_footprints = gpd.GeoDataFrame(pd.concat([images_filtered[['region', 'year', 'date', 'id']]
                                               .reset_index(drop = True),
                                               image_footprints],
                                    axis = 1))

area = pd.Series(dtype = 'float64')
crs_list = ['EPSG:32642', 'EPSG:32642', 'EPSG:32643', 'EPSG:32643']
for region in range(0, 4):
    image_subset = image_footprints[image_footprints.region == region]
    
    # set crs to appropriate UTM zone
    crs = crs_list[region]
    image_subset = image_subset.to_crs(crs)
    area = pd.concat([area, image_subset.geometry.area])
    
round(area.sum()/1e6/5000000*100, ndigits = 2)

In [ ]:
sns.displot(data = image_counts_f1, 
            x = 'max_cloud_cover',
#             hue = 'year',
#             multiple = 'stack',
#             alpha = 0.5,
           )